In [2]:
######################
#Datset Importing
#maybe save csv as list of dictionaries

import csv
import pandas as pd
import numpy as np
from IPython.display import display, HTML


TEST_PATH ='dataset/aps_failure_test_set.csv'
TRAIN_PATH ='dataset/aps_failure_training_set.csv'
SKIPROWS = 20



def loadDatasetWithCSV(path):
    with open(path, 'rt', encoding='utf-8') as csvfile:
        datareader = csv.reader(csvfile)
        for row in datareader:
            print(row)            

def loadDatasetWithPandas(path,skiprowsNum):
    #Reading the raw data from csv file
    rawData = pd.read_csv(path,skiprows=skiprowsNum)
    #display(rawData)
    #replacing the string indicating missing values with the numpy value for missing values
    naNProcessedData = rawData.replace({'na': np.nan}, regex=True)
    return naNProcessedData
    
    
    
#loadDatasetWithCSV('TEST_PATH')

#Load Datasets 
###############################################
#Load Train Dataset
trainData = loadDatasetWithPandas(TRAIN_PATH, SKIPROWS)
display(trainData)
#Load Test Dataset
testData = loadDatasetWithPandas(TEST_PATH, SKIPROWS)
#display(testData)

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,76698,NaN,2130706438,280,0,0,0,0,0,...,1240520,493384,721044,469792,339156,157956,73224,0,0,0
1,neg,33058,NaN,0,NaN,0,0,0,0,0,...,421400,178064,293306,245416,133654,81140,97576,1500,0,0
2,neg,41040,NaN,228,100,0,0,0,0,0,...,277378,159812,423992,409564,320746,158022,95128,514,0,0
3,neg,12,0,70,66,0,10,0,0,0,...,240,46,58,44,10,0,0,0,4,32
4,neg,60874,NaN,1368,458,0,0,0,0,0,...,622012,229790,405298,347188,286954,311560,433954,1218,0,0
5,neg,38312,NaN,2130706432,218,0,0,0,0,0,...,388574,288278,900430,300412,1534,338,856,0,0,0
6,neg,14,0,6,NaN,0,0,0,0,0,...,168,48,60,28,0,0,0,0,0,0
7,neg,102960,NaN,2130706432,116,0,0,0,0,0,...,715518,384948,915978,1052166,1108672,341532,129504,7832,0,0
8,neg,78696,NaN,0,NaN,0,0,0,0,0,...,699290,362510,1190028,1012704,160090,63216,41202,4,0,0
9,pos,153204,0,182,NaN,0,0,0,0,0,...,129862,26872,34044,22472,34362,0,0,0,0,0
